In [1]:
import pandas as pd
import datetime as dt
import pmaw

In [2]:
def keyword_filter(df, keywords):

    filtered_list = []
    for keyword in keywords:
        for text in df['text']:
            if keyword in text:
                filtered_list.append(text)

    filtered_df = pd.concat([df['datetime'], pd.DataFrame(filtered_list).rename(columns = {0: 'text'})], axis = 1).dropna()
    return(filtered_df)

In [3]:
def subreddit_fetch(subreddit, limit, after, before):

    comments_response = pushshift.search_comments(subreddit = subreddit, limit = limit, after = after, before = before)
    comments_original = pd.DataFrame(comments_response)
    comments_original['datetime'] = comments_original.apply(lambda row : dt.datetime.fromtimestamp(row['created_utc']), axis = 1)
    comments = comments_original[['datetime', 'body']].rename(columns = {'body': 'text'})
    
    return(comments)

In [4]:
pushshift = pmaw.PushshiftAPI()

In [5]:
after = int(dt.datetime(2012, 1, 1, 0, 0).timestamp())
before = int(dt.datetime(2021, 1, 1, 0, 0).timestamp())

wallstreetbets_comments = subreddit_fetch('wallstreetbets', limit = 100, after = after, before = before)
wallstreetbets_comments

Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.


,datetime,text
0,2013-10-19 13:58:39,I feel like a lot of people on wsb are going t...
1,2013-10-19 12:29:59,God I thought making 70% on American Airlines ...
2,2013-10-19 10:36:18,Someone needs to set that up. \n\nAnyone have ...
3,2013-10-19 08:56:48,That's awesome and I really hope he had inside...
4,2013-10-19 08:17:54,oh really?\n\nAAMRQ 10m avg \nFNMA 14.5m avg\n...
...,...,...
95,2013-10-17 16:23:42,"As I said, if you aren't comfortable trading.\..."
96,2013-10-17 16:22:09,So you are saying amrn has cash and a pipeline?
97,2013-10-17 16:21:09,All those stocks you mentioned did bounce to s...
98,2013-10-17 16:02:57,Thanks c0mputar I appreciate your contribution...


In [6]:
wallstreetbets_apple_comments = keyword_filter(wallstreetbets_comments, ['Apple', 'apple', 'AAPL'])
wallstreetbets_apple_comments

,datetime,text
0,2013-10-19 13:58:39,"Touche, and fair enough. Congrats on the smart..."
1,2013-10-19 12:29:59,&gt; Note: Yesterday I sold all of my position...
